# Implement VQE for H2 natively in braket SV1

In [1]:
# import required libraries
import os
import time
import numpy as np
from matplotlib import pyplot as plt
from openfermion import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpyscf import run_pyscf

from braket.circuits import Circuit, observables
from braket.devices import LocalSimulator
from braket.parametric import FreeParameter
from braket.aws import AwsDevice

## create hamiltonian

In [2]:
# create a directory named "data" to store intermediate classical computation results from OpenFermion
!mkdir -p "data"

In [3]:
print(f"INFO: Computing Hamiltonian for bond-length 0.735 A...")
geom = [('H', (0., 0., 0.)), ('H', (0., 0., 0.735))]
    # Make sure a directory named 'data' exists in the current folder,
    # else this statement will throw an error!
h2_molecule = MolecularData(
    geometry=geom, basis='sto-3g', multiplicity=1,
    description='bondlength', filename="",
    data_directory=os.getcwd()+'/data')
# Run PySCF to get molecular integrals, HF and FCI energies
h2_molecule = run_pyscf(molecule=h2_molecule, run_scf=True,
                        run_mp2=False, run_cisd=False,
                        run_ccsd=False, run_fci=True, verbose=False)


INFO: Computing Hamiltonian for bond-length 0.735 A...


In [4]:
# Convert electronic Hamiltonian to qubit operator using JW encoding
h2_qubit_hamiltonian = jordan_wigner(get_fermion_operator(
    h2_molecule.get_molecular_hamiltonian(occupied_indices=None,
                                              active_indices=None)))
# store molecular data and qubit operator for this config in an ordered list
mol_config = [h2_molecule, h2_qubit_hamiltonian, np.inf]

## create Ansatz

In [5]:
# Construct circuit for UCCSD ansatz parameterized by VQE parameter per McArdle et al. 
a_theta = FreeParameter("a_theta")
# Initialize HF state |0011>
ansatz_uccsd = Circuit().x(2).x(3)
# Perform initial rotations to measure in Y & X bases
ansatz_uccsd.rx(3,np.pi/2.).h(range(3))
# Entangle with CNOTs
ansatz_uccsd.cnot(3,2).cnot(2,1).cnot(1,0)
# Perform the rotation in Z-basis
ansatz_uccsd.rz(0,a_theta)
# Uncompute the rotations
ansatz_uccsd.cnot(1,0).cnot(2,1).cnot(3,2)
ansatz_uccsd.h(range(3))
ansatz_uccsd.rx(3,-np.pi/2.)

# initialize quantum device to run VQE over
local_sim = LocalSimulator()
device_sv1 = device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')   

# verify by running the circuit with no rotation and verifying we get HF state
print(ansatz_uccsd)
n_qubits = ansatz_uccsd.qubit_count
print(f"Total number of qubits in the circuit: {n_qubits}")
# We can fix the value of a_theta by calling the circuit with the value filled as follows.
# print(local_sim.run(ansatz_uccsd(0)).result().state_vector)
print(local_sim.run(ansatz_uccsd(0), shots=1000).result().measurement_probabilities)
# We can also fix the value of a_theta by supplying the inputs argument to run
print(local_sim.run(ansatz_uccsd, shots=1000, inputs={'a_theta': 0.0}).result().measurement_probabilities)

T  : │  0  │     1      │  2  │  3  │  4  │       5       │  6  │  7  │  8  │      9      │
      ┌───┐                          ┌───┐ ┌─────────────┐ ┌───┐ ┌───┐                     
q0 : ─┤ H ├──────────────────────────┤ X ├─┤ Rz(a_theta) ├─┤ X ├─┤ H ├─────────────────────
      └───┘                          └─┬─┘ └─────────────┘ └─┬─┘ └───┘                     
      ┌───┐                    ┌───┐   │                     │   ┌───┐ ┌───┐               
q1 : ─┤ H ├────────────────────┤ X ├───●─────────────────────●───┤ X ├─┤ H ├───────────────
      └───┘                    └─┬─┘                             └─┬─┘ └───┘               
      ┌───┐    ┌───┐     ┌───┐   │                                 │   ┌───┐     ┌───┐     
q2 : ─┤ X ├────┤ H ├─────┤ X ├───●─────────────────────────────────●───┤ X ├─────┤ H ├─────
      └───┘    └───┘     └─┬─┘                                         └─┬─┘     └───┘     
      ┌───┐ ┌──────────┐   │                                             │   ┌──

## Computation of expectation value of Hamiltonian operator for $H_2$

In [6]:
def calculate_observable_expectation(a_indices_gates, a_ckt, a_dev, a_shots):
    """ Calculates the expectation value of the given observable
    Parameters:
        a_indices_gates [sequence(tuple(int, str))]: List of tuple of qubit index & observable
        a_ckt [Circuit]: Braket circuit to which measurement gates are to be added
        a_dev [Braket Device]: Quantum device to run a_ckt on
        a_shots [int]: No. of shots for measuring in bases other than the computational basis
    Returns:
        float: The expectation value of the observable
    """
    if not a_indices_gates:
        # this is the constant term of the Hamiltonian
        return 1
    factors = {}
    for ind, factor in a_indices_gates:
        # N.B.: Convert from OpenFermion's little-endian convention to Braket's big-endian convention
        qubit = n_qubits - 1 - ind
        if factor == "X":
            factors[qubit] = observables.X()
        elif factor == "Y":
            factors[qubit] = observables.Y()
        elif factor == "Z":
            factors[qubit] = observables.Z()
    qubits = sorted(factors)
    observable = observables.TensorProduct([factors[qubit] for qubit in qubits])
    # initialize measuring circuit and add expectation measurement
    measuring_ckt = Circuit().add(a_ckt).expectation(observable=observable, target=qubits)
    # compute expectation value
    return a_dev.run(measuring_ckt, shots=a_shots).result().values[0]

def H_exp(a_qH, a_ckt, a_dev, a_shots=1000):
    """ Get expectation value of Hamiltonian for a given circuit result.
    Parameters:
        a_qH [OpenFermion QubitHamiltonian.terms]: Dictionary of OpenFermion QubitHamiltonian operator terms
        a_ckt [Braket Circuit]: Circuit to create the final state
        a_dev [Braket Device]: Quantum device to run a_ckt on
        a_shots [int]: No. of shots for measuring in bases other than the computational basis
    Returns:
        H_e [float]: Expectation value of a_qH for a_ket
    """
    # initialize expectation value of Hamiltonian
    H_e = 0.
    # loop over each term in qubit operator
    for term in a_qH:
        # extract the real-valued coefficient for this term
        coeff = np.real(a_qH[term])
        # compute and add this term's contribution to the Hamiltonian
        H_e += coeff * calculate_observable_expectation(term, a_ckt, a_dev, a_shots)
    return H_e

In [7]:
start = time.time()
# Loop over all VQE parameter values
for theta in np.linspace(start=-np.pi,stop=np.pi,num=24,endpoint=False):
    # get expectation value of this config's Hamiltonian for this parameter value
    exp_H_theta = H_exp(mol_config[1].terms, ansatz_uccsd(theta), device_sv1)
    # if this expectation value is less than min found so far, update it
    if exp_H_theta < mol_config[2]:
        # print(f"DEBUG: New optimum value of theta = {theta}")
        mol_config[2] = exp_H_theta

end = time.time()
print(f"min <H(R=0.735 A)> = {mol_config[2]:.4f} Ha")
print("Duration [sec]: ", (end-start))

min <H(R=0.735 A)> = -1.1376 Ha
Duration [sec]:  931.7416055202484
